In [1]:
import numpy as np
import gym
from numpy.random import choice
import random
from tensorbuilder.api import *
import tensorflow as tf

env = gym.make("FrozenLake-v0")

hdf5 not supported (please install/reinstall h5py)


[2017-01-26 12:15:35,761] Making new env: FrozenLake-v0


In [2]:
def next_action(actions, e):
    n = len(actions)
    
    if random.random() < e:
        return random.randint(0, n-1)
    else:
        return np.argmax(actions)
    
def naive_gd(error, expression):
    ws = tf.trainable_variables()
    gradients = tf.gradients(expression, ws)
    
    return [ tf.assign_add(w, error * g) for w, g in zip(ws, gradients) if w is not None ]

In [119]:
n_actions = env.action_space.n
n_states = env.observation_space.n
model_name = "shallow.model"
model_path = "/models/" + model_name
y = 0.95

graph = tf.Graph()
with graph.as_default():
    with tf.device("cpu:0"):
        s = tf.placeholder(tf.int32, [None], name='s')
        a = tf.placeholder(tf.int32, [None], name='a')
        r = tf.placeholder(tf.float32, [None], name='r')
        maxQs1 = tf.placeholder(tf.float32, [None], name='maxQs1')
        lr = tf.placeholder(tf.float32, [], name='lr')
        QSA = tf.placeholder(tf.float32, [None], name='QSA')

        ops = dict(trainable=True, weights_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01), biases_initializer=None) #tf.random_uniform_initializer(minval=0, maxval=0.01))


        Qs = Pipe(
            s,
            T.one_hot(n_states)
            .relu_layer(32, **ops)
            .linear_layer(4, scope='linear_layer', **ops),
            T
        )
        
        idx = tf.stack((tf.range(tf.shape(a)[0]), a), 1)
        Qsa = tf.gather_nd(Qs, idx)
        
        maxQs = tf.reduce_max(Qs, 1)
        
        
#         error = tf.placeholder(tf.float32, [])
        loss = Pipe(r + y * maxQs1 - Qsa, tf.nn.l2_loss, tf.reduce_sum)
        update3 = tf.train.GradientDescentOptimizer(lr).minimize(loss)
        
        error = lr * (r + y * maxQs1 - Qsa)
        update = naive_gd(error[0], Qsa[0])
        
        ws = tf.trainable_variables()
        dws = [ tf.placeholder(w.dtype, w.get_shape()) for w in ws ] 
        update2 = [ tf.assign_add(w, dw) for w, dw in zip(ws, dws) ]
        gradients = tf.gradients(Qsa[0], ws)

        writer = tf.summary.FileWriter('/logs/' +  model_name)
        saver = tf.train.Saver()

In [120]:
with tf.Session(graph=graph) as sess:
    with tf.device("cpu:0"):
        sess.run(tf.global_variables_initializer())

        _s = np.random.randint(0, 16)
        _a = np.random.randint(0, 4)
        _r = 3.2
        _maxQs1 = 2.3
        _lr = 0.5
        _qsa = 0.434

        L = 0

        W0 = sess.run(ws)
        WW = W0[L]

        print WW

        reset = lambda: sess.run([ tf.assign(w, dw) for w, dw in zip(ws, W0) ])

        _error, _ = sess.run([error[0], update], feed_dict={s: [_s], a: [_a], r: [_r], maxQs1: [_maxQs1], lr: _lr, QSA: [_qsa]})

        W1 = sess.run(ws)[L]
        print _error
        print "W1", W1 - WW


        reset()

        [_qs, _gradients] = sess.run([Qs, gradients], {s: [_s], a: [_a]})
        d = _lr * (_r + y * _maxQs1 - _qs[0,_a])
        _dws = [ d * dw for dw in _gradients ]

        sess.run(update2, feed_dict={
            dw: _dw for dw, _dw in zip(dws, _dws)
        })

        W2 = sess.run(ws)[L]

    #     sess.run(update2, feed_dict={s: [0], a: [1], r: [3.2], maxQs1: [2.3], lr: 0.5, QSA: [0.434]})
        print d
        print "W2", W2 - WW

        reset()

        _error, _ = sess.run([error[0], update3], feed_dict={s: [_s], a: [_a], r: [_r], maxQs1: [_maxQs1], lr: _lr, QSA: [_qsa]})

        W3 = sess.run(ws)[L]
        print _error
        print "W3", W3 - WW

[[  5.65255852e-03   8.46439693e-03   3.95233277e-03   9.35507752e-03
    9.17865057e-03   6.39708014e-04   7.97517039e-03   4.48554754e-04
    1.33927225e-03   7.39349332e-03   2.46193400e-03   4.95461561e-03
    5.83832711e-03   2.48491760e-05   6.16815686e-03   2.73495913e-04
    4.66776127e-03   1.24621147e-03   7.66848773e-03   8.25199392e-03
    4.48218221e-03   6.35457132e-03   2.13983166e-03   7.07066036e-04
    9.54912859e-04   6.21412508e-03   5.78961475e-03   6.78531639e-03
    5.66460844e-03   6.86134677e-03   8.62788502e-03   5.77358808e-03]
 [  1.69945718e-03   3.97772901e-03   2.90793763e-03   7.83976074e-03
    1.60481455e-03   2.05958844e-03   4.81131300e-03   3.66271357e-03
    2.44214409e-03   5.03819203e-03   4.99379728e-03   4.28048708e-03
    9.10380855e-03   7.60943396e-03   6.79889927e-04   1.83818571e-03
    5.94038703e-03   7.44578941e-03   9.15263966e-03   5.03937108e-03
    8.25659023e-04   5.06838411e-03   6.78669661e-03   1.02294201e-03
    9.88058932e-03 

In [121]:
with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as sess:
    with tf.device("cpu:0"):
        sess.run(tf.global_variables_initializer())
    #     saver.restore(sess, model_path)

        r_total = 0
        k = 2000.0

        for t in range(200000):
            _lr = 0.2 * k / (k + t)
    #         _lr = 0.5

            _s = env.reset()
            _qs = sess.run(Qs, feed_dict={s: [_s]})[0]

            while True:
    #             print _qs, _s
    #             print ""

                _a = next_action(_qs, 0.1)

                #calculate gradients
                _gradients = sess.run(gradients, {s: [_s], a: [_a]})

                #take step
                _s1, _r, done, info = env.step(_a)
                r_total += _r

                #calculate next Qs
                [_maxQs1, _qs1] = sess.run([maxQs, Qs], feed_dict={s: [_s1]})

                d = _r + y * _maxQs1 - _qs[_a]
                _dws = [ _lr * d * dw for dw in _gradients ]

#                 train
#                 sess.run(update2, feed_dict={
#                     dw: _dw for dw, _dw in zip(dws, _dws)
#                 })
                sess.run(update3, feed_dict={
                    s: [_s], a: [_a], r: [_r], 
                    lr: _lr * 3.0,
                    maxQs1: np.max(_qs1, 1),
                    QSA: [_qs[_a]]
                })

                #update state
                _s = _s1
                _qs = _qs1[0]

                if done:
                    break

            if t % 500 == 0:
                print r_total, "of", 500, ", lr:", _lr
                r_total = 0
                saver.save(sess, model_path)


0.0 of 500 , lr: 0.2
3.0 of 500 , lr: 0.16
0.0 of 500 , lr: 0.133333333333
0.0 of 500 , lr: 0.114285714286
0.0 of 500 , lr: 0.1
0.0 of 500 , lr: 0.0888888888889
0.0 of 500 , lr: 0.08
0.0 of 500 , lr: 0.0727272727273
11.0 of 500 , lr: 0.0666666666667
9.0 of 500 , lr: 0.0615384615385
23.0 of 500 , lr: 0.0571428571429
14.0 of 500 , lr: 0.0533333333333
16.0 of 500 , lr: 0.05
11.0 of 500 , lr: 0.0470588235294
15.0 of 500 , lr: 0.0444444444444
16.0 of 500 , lr: 0.0421052631579
8.0 of 500 , lr: 0.04
20.0 of 500 , lr: 0.0380952380952
23.0 of 500 , lr: 0.0363636363636
22.0 of 500 , lr: 0.0347826086957
29.0 of 500 , lr: 0.0333333333333
24.0 of 500 , lr: 0.032
26.0 of 500 , lr: 0.0307692307692
26.0 of 500 , lr: 0.0296296296296
20.0 of 500 , lr: 0.0285714285714
17.0 of 500 , lr: 0.0275862068966
21.0 of 500 , lr: 0.0266666666667
27.0 of 500 , lr: 0.0258064516129
25.0 of 500 , lr: 0.025
34.0 of 500 , lr: 0.0242424242424
36.0 of 500 , lr: 0.0235294117647
37.0 of 500 , lr: 0.0228571428571
45.0 of 500 

KeyboardInterrupt: 

In [28]:
_s = env.reset()
done = False
with tf.Session(graph=graph) as sess:
    saver.restore(sess, model_path)
    
    for i in range(100):
        _qs = sess.run(Qs, feed_dict={s: [_s]})
        _a = next_action(_qs, 0)
        _s, _r, done, info = env.step(_a)
        env.render()
        print("")

        if done:
            print(_r)
            break

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Right)

SFFF
FHFH
FFFH
HFFG
  (Down)

SFFF
FHFH
FFFH
HFFG
  (Down)

SFFF
FHFH
FFFH
HFFG
  (Down)

SFFF
FHFH
FFFH
HFFG
  (Down)

0.0
